In [ ]:
import pandas as pd
import numpy as np


def calculate_sortino_ratio(
    returns: pd.Series, risk_free_rate: float = 0.0, annualization_factor: int = 365
):
    """
    Calculate the annualized Sortino Ratio from a pandas Series of returns.

    Parameters:
    - returns: pd.Series of periodic returns (e.g. daily)
    - risk_free_rate: optional, in same period units as returns (e.g. 0.0 for daily, 0.01/252 for 1% annual)
    - annualization_factor: 252 for daily, 12 for monthly, etc.

    Returns:
    - Annualized Sortino Ratio (float)
    """
    # Excess returns
    excess_returns = returns - risk_free_rate

    # Downside deviation (only negative excess returns)
    downside_returns = excess_returns[excess_returns < 0]
    downside_deviation = np.std(downside_returns, ddof=0)

    # Mean excess return
    mean_excess_return = excess_returns.mean()

    # Raw Sortino Ratio
    if downside_deviation == 0:
        return np.inf if mean_excess_return > 0 else -np.inf

    sortino = mean_excess_return / downside_deviation

    # Annualize
    return sortino * np.sqrt(annualization_factor)

In [ ]:
df = pd.read_csv("csv_path")
df

In [ ]:
rolling_window = 100
# mean = df['close_pi'].mean()
df["mean"] = df["close_pi"].rolling(window=rolling_window).mean()
df["std"] = df["close_pi"].rolling(window=rolling_window).std()
df["zscore"] = (df["close_pi"] - df["mean"]) / df["std"]
df

In [ ]:
threshold = 1
zscore = df["zscore"].values
len(zscore)

In [ ]:
# assign pos array with initial pos = 0
pos = [0]
long_trades = 0
short_trades = 0
# loop all zscore values
for i in range(1, len(zscore)):
    # current zscore more than threshold , pos = 1
    if zscore[i] >= threshold:
        pos.append(1)
        if pos[-2] != pos[-1]:
            long_trades += 1
    # current zscore less than negative theshold , pos = -1
    elif zscore[i] <= -threshold:
        pos.append(-1)
        if pos[-2] != pos[-1]:
            short_trades += 1
    # current zscore within theshold and negative threshold , pos = previous pos
    else:
        pos.append(pos[-1])

df["pos"] = pos
df

In [ ]:
# calculate return
df["return"] = df["close"].pct_change()
# get the previous pos to a new column
df["pos_shift"] = df["pos"].shift(1)
# calculate trades
df["trades"] = abs(df["pos"] - df["pos_shift"])
# calculate pnl
fees = 0.06
df["pnl"] = df["pos_shift"] * df["return"] - df["trades"] * fees / 100
df

In [ ]:
df["cumu"] = df["pnl"].cumsum()
df["dd"] = df["cumu"] - df["pnl"].cummax()

sr = df["pnl"].mean() / df["pnl"].std() * np.sqrt(365 * 24)
cr = df["pnl"].mean() / abs(df["dd"].min())
sortino_ratio = calculate_sortino_ratio(df["pnl"])
number_of_trades = df["trades"].sum()
ar = df["pnl"].mean() * 365

In [ ]:
report = {
    "Rolling Window": rolling_window,
    "Threshold": threshold,
    "SR": sr,
    "CR": cr,
    "AR": ar,
    "MDD": df["dd"].min(),
    "Sortino Ratio": sortino_ratio,
    "Number of Trades": number_of_trades,
}
report